In [37]:
import keras
import numpy as np
# Nietzche
# path = keras.utils.get_file(
#     'nietzsche.txt',
#     origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')

# Bras cubas
path = './dataset/memoriasBras.txt'

text = open(path).read().lower()
print('Corpus length:', len(text))

Corpus length: 354995


In [38]:
# Normalizing text
normalized_text = text.replace('\n', ' ').replace('preface   ', '').replace('  ', ' ').replace('  ', ' ').replace('  ', ' ').replace('  ', ' ').replace('  ', ' ').replace('  ', ' ').replace('  ', ' ').replace('  ', ' ').replace('\x0c', '').replace('ü','u')
normalized_text[0:300]

'memórias póstumas de brás cubas texto-fonte: obra completa, machado de assis, rio de janeiro: editora nova aguilar, 1994. publicado originalmente em folhetins, a partir de março de 1880, na revista brasileira. ao verme que primeiro roeu as frias carnes do meu cadáver dedico como saudosa lembrança es'

In [12]:
maxlen = 60
step = 3
sentences = []
next_chars = []

for i in range(0, len(normalized_text) - maxlen, step):
    sentences.append(normalized_text[i: i + maxlen])
    next_chars.append(normalized_text[i + maxlen])
print('Number of sequences:', len(sentences))

chars = sorted(list(set(normalized_text)))
print('Unique characters:', len(chars))

char_indices = dict((char, chars.index(char)) for char in chars)
print('Vectorization...')

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 118211
Unique characters: 72
Vectorization...


In [13]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               102912    
_________________________________________________________________
dense_1 (Dense)              (None, 72)                9288      
Total params: 112,200
Trainable params: 112,200
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Add some temperature to the predictions for more uncommon predictions
def sample(preds, temperature=1.0):
    # Add some temperature to the prediction array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    # "Rolls a dice" with the probabilities in the array and sets the chosen index to 1 and others to 0
    probas = np.random.multinomial(1, preds, 1)
    # Returns the chosen index 
    return np.argmax(probas)


In [23]:
import random

def get_next_char(prev, preds, chars):
    if prev == ' ' and round(random.random()):
        char = chr(random.randint(97,122))
        while char == 'w' or char == 'y' or char == 'x':
            char = chr(random.randint(97,122))
    else:
        char = chars[np.argmax(preds)]
    return char

In [18]:
import sys

for epoch in range(1, 60):
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)
    start_index = random.randint(0, len(normalized_text) - maxlen - 1)
    generated_text = normalized_text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            generated_text += next_char
            generated_text = generated_text[1:]
            sys.stdout.write(next_char)

epoch 1
924/924 [==============================] - 220s 238ms/step - loss: 1.9141
--- Generating with seed: "esus. e não me deixou sem provar que o apólogo evangélico nã"
------ temperature: 0.2
esus. e não me deixou sem provar que o apólogo evangélico não sentimento de alma de um com a constar de um dessentimante de um como de um a constar de um a minha a menhar de um estava a menhar de um comigo de um dessentimento de um desconter a menhar de um constar da marente de um constante de um a menhar de um com a maria de um a carecia de um de um comigo de um comentas de um despossas de um esta de a menhar a casar a menhar de um como e a menhar a menha------ temperature: 0.5
a de a menhar a casar a menhar de um como e a menhar a menha estava marando de o capítulo xxiiii / a menos de morres a respontivela de um a casar de antar o outro mentos de costar estava minha a o tardo os constas, e a menhor de um almar desandeira de assentimento e como em parei o consalho mentar a comição na corrar o 

KeyboardInterrupt: 

In [26]:
# Only train model
model.fit(x, y, batch_size=128, epochs=10)

Epoch 1/10
924/924 [==============================] - 165s 179ms/step - loss: 1.5960
Epoch 2/10
154/924 [====>.........................] - ETA: 2:35 - loss: 1.5114

KeyboardInterrupt: 

In [27]:
# Testing new sampling method: getting a random character sometimes only when starting a new word
size = 300

start_index = random.randint(0, len(normalized_text) - maxlen - 1)
generated_text = normalized_text[start_index: start_index + maxlen]
print('--- Generating with seed: "' + generated_text + '"')

for i in range(size):
    sampled = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(generated_text):
        sampled[0, t, char_indices[char]] = 1.
    preds = model.predict(sampled, verbose=0)[0]
    next_char = get_next_char(next_char, preds, chars)
    generated_text += next_char
    generated_text = generated_text[1:]
    print(next_char, end='')

--- Generating with seed: " é o saldo que o deficit. como era muito seco de maneiras ti"
ramos um insterno de um nomeito de porta justante e jura de minha olhar a almoção por isso de um outro homem de minha baixa interração virgília de um contos, e estava a carta de interrostiva de um capítulo. repois resporta de um capítulo. não era kembra sentir a consciência a zer a juria de lançava 

In [57]:
# Getting words from Bras Cubas
special_tokens = ['.', ',', ':', ';', '!', '?', '(', ')', '“', '”']
simplified_text = normalized_text
for t in special_tokens:
    simplified_text = simplified_text.replace(t, '')
words = list(set(simplified_text.split(' ')))
unwanted_words = [chr(x) for x in range(97,123)] + ['']
for w in words:
    for uw in unwanted_words:
        if w == uw:
            words.remove(uw)
len(words)

10488

In [55]:
a = [1,2]
# a.concat([1,2])
a + a

[1, 2, 1, 2]